Пример кода взят из книги: "Глубокое обучение" под авторством Николенко С., Кадурин А., Архангельская Е. с 87

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
n_samples = 1000
batch_size = 100
num_steps = 20000
# Создаем входные данные, 1000 случайных точек для предиктора [0;1]
# И для каждого значения предиктора расчитываем "почти" правильный ответ
# y = 2x + 1 + eps 
x_data = np.random.uniform(1, 10, (n_samples, 1))
y_data = 2 * x_data + 1 + np.random.normal(0, 2, (n_samples, 1))

In [ ]:
# Так называемая заглушка, используемая для передачи новых данных на видеокарту
# Хочет знать тип данных и размер тензора
x = tf.placeholder(tf.float32, shape=(batch_size, 1))
y = tf.placeholder(tf.float32, shape=(batch_size, 1))

In [ ]:
# Инициализация переменных (на данный момент значений не имеют)
with tf.variable_scope('linear-regression'):
    k = tf.Variable(tf.random_normal((1, 1)), name='slope')
    b = tf.Variable(tf.zeros((1,)), name='bias')

In [ ]:
# Тут мы задаем нашу модель, выбираем целевую функцию
y_pred = tf.matmul(x, k) + b
loss = tf.reduce_sum((y - y_pred) ** 2)
# Собственно выбор алгоритма, который будет подсчитывать градиенты
# И обновлять веса
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000002).minimize(loss)

In [ ]:
"""
Цикл в котром происходит вся магия.
На каждой итерации цикла берем случайное подмножество размера batch_size
И соственно обучем на этих примерах
"""
display_step = 100
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(num_steps):
    indices = np.random.choice(n_samples, batch_size)
    x_batch = x_data[indices]
    y_batch = y_data[indices]
    _, loss_val, k_val, b_val = sess.run([optimizer, loss, k, b], feed_dict = {x : x_batch, y : y_batch})   
    if (i+1) % display_step == 0:
        print('Эпоха %d: %.8f, k=%.4f, b=%.4f' % (i + 1, loss_val, k_val, b_val))

# Keras

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, Activation

In [4]:
# Создаем модель в виде sequential т.е. модель будеть создаваться слой за слоем
logr = Sequential()
# Добавляем один слой, входы которого будут размерности два, а активация - сигмоида
logr.add(Dense(1, input_dim=2, activation='sigmoid'))
# Компилируем модели с такими вот параметрами, все очевидно
logr.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
# Для примера гегерируем два двумерных нормальных распределения.
def sampler(n, x, y):
    return np.random.normal(size=[n, 2]) + [x, y]
# С центром в (-1, -1) и (1, 1) и дичперсией 1 по обоим компонентам
def sample_data(n=1000, p0=(-1., -1.), p1=(1., 1.)):
    zeros, ones = np.zeros((n, 1)), np.ones((n, 1))
    labels = np.vstack([zeros, ones])
    z_sample = sampler(n, x=p0[0], y=p0[1])
    o_sample = sampler(n, x=p1[0], y=p1[1])
    return np.vstack([z_sample, o_sample]), labels

x_train, y_train = sample_data()
x_test, y_test = sample_data(100)

logr.fit(x_train, y_train, batch_size=16, epoch=100, verbose=1, validation_data=(x_test, y_test))